## Open notebook in:
| Colab                                 |  Gradient                                                                                                                                         |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jyellow/Transformers-in-Action/blob/main/CH03/ch03_text_summarization.ipynb)                                              | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH03/ch03_text_summarization.ipynb)|             

# Download repo

In [ ]:
# Clone repo, if it's not already cloned, to be sure all runs on smoothly
# on Colab, Kaggle or Paperspace
import os

if not os.path.isdir('Transformers-in-Action'):
    !git clone https://github.com/jyellow/Transformers-in-Action.git
else:
    print('Repository already exists. Skipping clone.')


current_path = %pwd
if '/Transformers-in-Action' in current_path:
    new_path = current_path + '/utils'
else:
    new_path = current_path + '/Transformers-in-Action/utils'
%cd $new_path

# About this notebook

Welcome to this exciting notebook, where we dive into the world of text summarization using a variety of cutting-edge models and techniques. In this notebook, we will put several state-of-the-art Transformer-based models, including BART, T5, ProphetNet and Pegasus to the test. As a baseline, we will also explore a TextRank-based model to compare their performance. Our objective is to analyze the efficacy and suitability of these models in generating summaries for different types of datasets, such as [CNN daily](https://huggingface.co/datasets/cnn_dailymail) news articles and [arXiv scientific papers](https://huggingface.co/datasets/scientific_papers).
To further challenge these models, we will utilize the BigBird and Longformer architectures, which are specifically designed to handle longer input sequences. This will enable us to examine the models' performance on the CNN daily and arXiv datasets, which consist of diverse content and varying text lengths.
In this notebook, you can expect to:
1. Gain an introduction to the various text summarization models, including their key features and limitations.
2. Learn how to implement and utilize these models to generate summaries for different types of text.
3. Assess the models' performance on multiple samples from the CNN daily and arXiv datasets using both ROUGE and BLEU metrics for an in-depth comparison.
4. Understand the significance of evaluating models on a diverse set of samples to achieve a more dependable and generalizable performance evaluation.
5. Discover the challenges associated with assessing models on complex and domain-specific datasets, and how the BigBird and Longformer architectures can help address these issues.

By the end of this notebook, you will have a comprehensive understanding of various text summarization models, their performance on different datasets, and the intricacies involved in evaluating models on complex, domain-specific data. Let's embark on this fascinating journey and unlock the potential of these powerful text summarization models!

# Install requirements

In [ ]:
!pip install -U tensorflow==2.12.0 -q
!pip install -U keras==2.12.0 -q
!pip install -U tensorflow-gcs-config==2.12.0 -q
!pip install -U tensorflow-probability==0.20.1 -q

In [3]:
from requirements import *

In [ ]:
install_base_packages()
install_required_packages_ch03()

# Imports

In [ ]:
from setup import *
from utils import *
from textrank import *

In [ ]:
useGPU()

In [7]:
import pandas as pd
import numpy as np
from functools import partial

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import networkx as nx

from summa import summarizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, set_seed
from transformers import LEDForConditionalGeneration, LEDTokenizer
from transformers import BartConfig, BartForConditionalGeneration
from transformers import LEDConfig, LEDForConditionalGeneration
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
from datasets import load_dataset

from multiprocessing import Pool
import evaluate
from tqdm import tqdm

import matplotlib.pyplot as plt

from textwrap import TextWrapper

set_seed(42)

In [8]:
# %%capture
# !pip install watermark

In [ ]:
# %load_ext watermark
# %watermark -a "Nicole Koenigstein" -gu "Nicolepcx" -u -d -v -m

# Allocate enough RAM

Let us try to get a __GPU__ with at least __15GB RAM__ for our notebook.

In [ ]:
# crash colab to get more RAM -> uncomment to use
# !kill -9 -1

We can execute the following command `!free -h`  to see if we have enough RAM and `!nvidia-smi` to get more info about our GPU type we got assigned.
If the allocated GPU is too small, the above cell can be used to run the command to crash the notebook hoping to get a better GPU after the crash, since the GPU is randomly allocated.


In [ ]:
!free -h

In [ ]:
!nvidia-smi

# TextRank Algorithmus

The custom `TextRank` class is designed to create a graph-based representation of a given text, where each node in the graph corresponds to a sentence, and the edges between nodes represent the similarity between sentences. This graph-based representation provides an intuitive way to visualize and understand the relationships among sentences in the text.

Using a graph to represent text is particularly useful for tasks like text summarization, where identifying the most important and representative sentences is crucial. By representing the text as a graph, the TextRank algorithm can leverage the graph structure to rank sentences based on their importance in the overall text.

TextRank is an unsupervised graph-based ranking algorithm inspired by Google's PageRank algorithm. It was first introduced in the paper ["TextRank: Bringing Order into Texts"](https://aclanthology.org/W04-3252/) by Mihalcea and Tarau (2004). The TextRank class in the code below constructs a graph by calculating the cosine similarity between sentences using the Bag of Words (BoW) and TF-IDF representation of the sentences. The higher the similarity, the stronger the connection between the sentences in the graph. By plotting this graph, we can visually explore the relationships among sentences and better understand the structure and organization of the text.

In [12]:
# We use the first paragraph for the original Transformer paper
# "Attention Is All You Need"
text = "Recurrent neural networks, long short-term memory [13] and gated recurrent [7] \
neural networks in particular, have been firmly established as state of the art approaches \
in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous \
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]. \
Recurrent models typically factor computation along the symbol positions of the input and output \
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden \
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently \
sequential nature precludes parallelization within training examples, which becomes critical at longer \
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved \
significant improvements in computational efficiency through factorization tricks [21] and conditional \
computation [32], while also improving model performance in case of the latter. The fundamental \
constraint of sequential computation, however, remains. \
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models \
in various tasks, allowing modeling of dependencies without regard to their distance in \
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms \
are used in conjunction with a recurrent network. \
In this work we propose the Transformer, a model architecture eschewing recurrence and instead \
relying entirely on an attention mechanism to draw global dependencies between input and output. \
The Transformer allows for significantly more parallelization and can reach a new state of the art in \
translation quality after being trained for as little as twelve hours on eight P100 GPUs."

In [ ]:
tr = TextRank()
sentences = tr.split_sentences(text)
G = tr.create_graph(sentences)
tr.plot_graph(G)

This plot shows a circular graph where each node represents a sentence from the input text. The edges connecting the nodes represent the similarity between the sentences, which is calculated using cosine similarity based on the TF-IDF representation of the sentences.

The numbers displayed on the edges are the weights of the edges, representing the cosine similarity scores between the connected sentences. These scores range from 0 to 1, where 0 indicates no similarity and 1 indicates a perfect match (identical sentences). A higher number implies a stronger connection between the two sentences, meaning they share more words or terms in common.

This graphical representation of the text allows us to visually analyze the relationships between the sentences in the input text. Which can help identify clusters of similar sentences or detect isolated sentences that might be less related to the main topics discussed in the text.

## TextRank Example from Book

 Text is represented as a graph, with nodes representing textual units (sentences or words) and edges representing the relationships between them.
The scores for each node are calculated on the basis of the scores of its neighbors and the weights of the edges that connect them. First, the algorithm applies the PageRank equation as an iteration:

\begin{equation}
p_i^{(k+1)}=\frac{1-\delta}{N}+\delta \cdot \sum_{\text {edges }_{j i}} \frac{p_j^{(k)}}{c_j}
\end{equation}

Then, as a second step, the PageRank is replaced by the rank $r$ of the vertices $V_{i}$, which can stand for words, phrases or complete sentences.

\begin{equation}
r\left(V_i^{(k+1)}\right)=\frac{1-\delta}{N}+\delta \cdot \sum_{\text {edges }_{j i}} \frac{\omega_{j i} \cdot r\left(V_j^{(k)}\right)}{\sum_{\text {edges }_{ji}} \omega_{j i}}
\end{equation}

Additionally, to each edge a weight $w_{ji}$ is attached in order to have more flexibility to characterize the \emph{strength} of the connection between two vertices $V_j$ and $V_i$. Depending on the use case, these edges can represent different types of connection, lexical relations, or contextual relations.

The pagerank function `def pagerank()` from the class `class TextRank` reflects the second equation, where $\omega_{j i}$ is the weight of the connection between vertices $V_j$ and $V_i$ (i.e., adjacency\_matrix[j, i]). The function calculates the rank $r(V_i^{(k+1)})$ for each vertex $V_i$ in the graph, considering the edge weights and the out-degree of each node.

In [ ]:
text = """
    Machine learning is a subfield of artificial intelligence.
    Deep learning is a technique used in machine learning.
    Supervised learning is a popular machine learning technique.
    Unsupervised learning is another machine learning technique.
    """

tr = TextRank()
sentences = tr.split_sentences(text)
G = tr.create_graph(sentences)
tr.print_sentence_weights(G, sentences)
tr.plot_graph(G)

# Use case CNN/Daily mail dataset

The [CNN/Daily Mail dataset](https://huggingface.co/datasets/cnn_dailymail) is a popular choice for evaluating text summarization models in the NLP community. This dataset consists of news articles from CNN and Daily Mail websites, along with their human-generated summaries, also known as "highlights" or "story points." It provides a large and diverse set of real-world articles and their corresponding summaries, making it an excellent benchmark for testing the performance of various summarization models.

The dataset is interesting as an example use case for several reasons:

1. Real-world data: The articles and summaries are sourced from actual news websites, which makes the dataset representative of the kind of content users might encounter and want to summarize in real-life scenarios.

2. Diversity: The dataset covers a wide range of topics, such as politics, sports, entertainment, and technology. This diversity helps evaluate how well a summarization model can generalize across different domains and subject matters.

3. Abstractive summarization: The human-generated highlights often involve rephrasing and paraphrasing of the original content, which requires a deeper understanding of the text. This makes the dataset suitable for testing the capabilities of abstractive summarization models, which aim to generate summaries that capture the main points of the input text while using new, concise, and coherent sentences.

4. Large-scale: With over 300,000 articles and summaries, the CNN/Daily Mail dataset is large enough to train and evaluate deep learning models, such as Transformer-based models, which often require substantial amounts of data to achieve high performance.

By using the CNN/Daily Mail dataset as an example use case, we can evaluate various Transformer-based summarization models on a challenging, diverse, and real-world dataset, which will give us valuable insights into their performance and potential applications.

## Load the CNN/Daily mail dataset

In [ ]:
# %%capture
# # install new version datasets package
# %pip install -U datasets

In [3]:
# import datasets
# datasets.__version__
# from datasets import load_dataset

In [16]:
# Load the CNN/Daily mail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

In [ ]:
dataset_train = load_dataset('cnn_dailymail', '3.0.0', split='train')
dataset_train.set_format(type='pandas', columns=['article', 'highlights'])

In [10]:
# Convert to pandas DataFrame
df = dataset_train.to_pandas()

## Excerpt of article and ground truth of summary




In [ ]:
# Access first row using iloc
sample = df.iloc[0]

excerpt = 1000

print(f"\033[1mExcerpt of {excerpt} characters, total length of article: \
{len(sample['article'])}:\033[0m\n")

print(sample["article"][:excerpt])
print(f"\033[1m\n\nSummary (length: {len(sample['highlights'])}):\033[0m\n")
print(sample["highlights"])


In [ ]:
import numpy as np
articles_np= np.array(dataset_train['article'])
articles_np.shape

In [ ]:
articles_np_1 = articles_np[:1]
articles_np_1.shape

In [ ]:
articles_np_1

## Establishing a baseline:
### Applying TextRank algorithm to CNN/Daily mail dataset


TextRank is a good choice as a baseline algorithm for text summarization because it is unsupervised, easy to implement, and computationally efficient. It provides a simple yet effective way to extract key phrases and rank sentences based on their importance within the text. Compared to other baseline choices, such as lead-3 (selecting the first three sentences) or simple frequency-based methods, TextRank offers a more sophisticated approach by capturing the relationships between sentences and their relevance to the overall text, giving a better starting point for comparison with advanced methods like Transformer-based models.

This code snippet is used to generate summaries for a collection of texts (in this case, news article) using the TextRank algorithm. The TextRank summarization is implemented with the summa library, which is available on [PyPI](https://pypi.org/project/summa/). The implementation is based on the paper Mihalcea, R., Tarau, P.: [“TextRank: Bringing order into texts”](https://aclanthology.org/W04-3252/)

A `summarize_text` function is defined, which takes in two arguments - text and words. text is the input text to be summarized, and words is an optional parameter to specify the target number of words in the generated summary (default is 250 words). The `summarizer.summarize()` function from the summa library is used to generate the summary.

The code then employs the multiprocessing Pool class to parallelize the summarization process. This speeds up the computation by taking advantage of multiple CPU cores available on the system. The `Pool()` context manager creates a pool of worker processes, and the `pool.map()` function is used to apply the `summarize_text` function to each element of the `articles_np_1` array (which contains the input articles).

The `pool.map()` function distributes the articles across the available worker processes, and each process applies the summarize_text function independently. Once all the summaries are generated, they are collected and stored in the `summarized_articles` list. This parallelization can significantly improve the performance of the TextRank summarization, especially when working with large datasets.


In [19]:
# from summa import summarizer
# from multiprocessing import Pool

# Define the function to summarize the text
def summarize_text(text, words=80):
    summary = summarizer.summarize(text, words=words)
    return summary



# Parallelize the TextRank summarization
with Pool() as pool:
    summarized_articles = pool.map(summarize_text, articles_np_1)


### Get summarized article

In [ ]:
summarized_articles

In [21]:
references = np.array(dataset_train['highlights'])

In [ ]:
references_np = np.array(references)
references_np[:1]

In [23]:
reference = references[0]

In [ ]:
print(summarized_articles[0])

In [25]:
summaries = {}
summaries['TextRank (Baseline)'] = summarized_articles

In [ ]:
print(summaries['TextRank (Baseline)'][0])

# Appyling Transformers to the CNN/Daily mail dataset

In this section, we define a function `generate_summary` that takes a `model_name` and a `sample_text` as input and generates a summary of the input text using the specified model. We support multiple summarization models such as:
- BART [Modelcard](https://huggingface.co/facebook/bart-large-cnn), [Paper](https://arxiv.org/abs/1910.13461)
- T5 [Modelcard](https://huggingface.co/sysresearch101/t5-large-finetuned-xsum-cnn), [Paper](https://arxiv.org/abs/1910.10683)
- ProphetNet [Modelcard](), [Paper](https://arxiv.org/abs/2001.04063)
- Pegasus [Modelcard](https://huggingface.co/google/pegasus-cnn_dailymail), [Paper](https://arxiv.org/abs/1912.08777)
- GPT-2 [Modelcard](https://huggingface.co/gpt2-xl), [Paper](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

The function sets up the tokenizer and model based on the given model_name and uses the Hugging Face's pipeline to perform the summarization.

For the CNN/Daily Mail dataset use case, we aim to evaluate different transformer models to understand their performance in summarizing news articles. The dataset contains long news articles with associated human-generated summaries, making it a great resource for benchmarking and comparing various summarization models.

By using multiple models for summarization, we can gain insights into the strengths and weaknesses of each approach, and understand how they perform in a real-world scenario. This helps us to identify the most suitable model for our specific use case, considering factors such as quality, speed, and resource requirements.

The choice of using the Hugging Face's pipeline function allows for a streamlined and easy-to-use interface for handling pre-processing, model inference, and post-processing steps. This simplifies the code and makes it more maintainable and user-friendly, especially for those who may not have extensive experience with transformers or NLP.

In [28]:
from transformers import set_seed

set_seed(42)

def generate_summary(model_name, sample_text):
    """
    Generate a summary of the input text using a specified model.

    Args:
        model_name (str): The name of the model to use for summarization. Supported values are "BART", "T5",
            "ProphetNet".
        sample_text (str): The input text to summarize.

    Returns:
        str: The summary generated by the specified model.

    Raises:
        ValueError: If the specified model is not supported.


    Example:
        >>> generate_summary("BART", "This is an example text to summarize using BART.")
        'BART is used to summarize the input text.'
    """

    # Set up tokenizer and model
    model_dict = {
        "BART": "facebook/bart-large-cnn",
        "T5": "sysresearch101/t5-large-finetuned-xsum-cnn",
        "ProphetNet": "microsoft/prophetnet-large-uncased-cnndm"
    }

    if model_name in model_dict.keys():
        tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name])
        model = AutoModelForSeq2SeqLM.from_pretrained(model_dict[model_name])
        summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model)
        # Generate summary using the specified model
        summarization_output = summarization_pipeline(sample_text, max_length=100)
        summary = summarization_output[0]["summary_text"]
    else:
        raise ValueError(f"Model {model_name} is not supported.")

    return summary


## Generate summary from  <font color='red'>"BART"</font> model

In [ ]:
summaries["BART"] = generate_summary("BART", sample_text)
summaries["BART"]

## Generate summary from  <font color='red'>"T5"</font> model

In [ ]:
summaries["T5"] = generate_summary("T5", sample_text)
summaries["T5"]

## Generate summary from  <font color='red'>"ProphetNet"</font> model

In [ ]:
summaries["ProphetNet"] = generate_summary("ProphetNet", sample_text)

## Evaluate all models incl. baseline on CNN/Daily mail dataset

### Evaluating Summarization Models using ROUGE and BLEU Metrics

In this section, we demonstrate how to evaluate the summarization models using both the ROUGE and BLEU metrics. We use the Hugging Face's implementation of these metrics.

ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation. Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.
[Link](https://huggingface.co/spaces/evaluate-metric/rouge)

BLEU, or Bilingual Evaluation Understudy, is an evaluation metric used primarily for machine translation. It measures the similarity between a candidate translation and a set of reference translations, considering both the n-gram precision and a brevity penalty factor.
[Link](https://huggingface.co/spaces/evaluate-metric/google_bleu)

[Info about Huggingface evaluate](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/loading_methods#evaluate.load)
<br> <br>

__Note:__ It can be beneficial to use both ROUGE and BLEU metrics for evaluation, as they focus on different aspects of the generated summaries. ROUGE is recall-oriented and mainly focuses on the coverage of important content, while BLEU is precision-oriented and evaluates the fluency and correctness of the generated text.

It is important to have an understanding of what constitutes a "good" score when evaluating summarization models. Here are some guidelines for interpreting the ROUGE and BLEU scores:

__ROUGE Scores__

ROUGE scores usually range from 0 to 1, with 1 being a perfect match between the generated summary and the reference summary. In practice, scores closer to 1 are rare, especially for abstractive summarization tasks.

For a simple baseline model like TextRank or LSA, a "good" ROUGE score can be around 0.3 to 0.4. For more advanced Transformer-based models like BART, T5, or Pegasus, a "good" ROUGE score can range from 0.4 to 0.6, depending on the task and dataset. State-of-the-art models can achieve even higher scores, but it's important to keep in mind that ROUGE scores are just one aspect of evaluating the quality of generated summaries.

There exist different variants of ROUGE, which consider different aspects of the summaries, such as n-grams and the longest common subsequence.


__BLEU Scores__

BLEU scores also range from 0 to 1, with 1 indicating a perfect match between the generated text and the reference text. However, BLEU scores tend to be lower compared to ROUGE scores, as they are more sensitive to the differences in word order and phrasing.

For a simple baseline model, a "good" BLEU score could be around 0.1 to 0.2. For Transformer-based models, a "good" BLEU score might range from 0.2 to 0.4, again depending on the task and dataset. State-of-the-art models can achieve higher scores, but just like with ROUGE, it's essential to consider other factors when evaluating the quality of generated summaries.

Keep in mind that these are rough guidelines, and the definition of a "good" score can vary depending on the specific domain, dataset, and evaluation criteria. It is always a good idea to compare your model's scores with scores from existing state-of-the-art models on the same dataset to get a better understanding of its performance.


__Defining the SummarizationMetrics Class__

The SummarizationMetrics class is defined with methods for computing the ROUGE and BLEU metrics, as well as a method to compute both metrics together:

`class SummarizationMetrics:
    ...
`

This class is located in the `utils.py` file, feel free to look it up to get a better understanding how the mectrics are computed.

__Computing Metrics__

To compute the metrics, we first instantiate the SummarizationMetrics class:
`evaluator = SummarizationMetrics()
`

Then, we call the compute_sum_metric method on the evaluator object, passing the generated summaries and the reference summary:
`metrics_df = evaluator.compute_sum_metric(summaries, reference)
`

The compute_sum_metric method computes both the ROUGE and BLEU metrics by calling the `compute_rouge_metrics` and `compute_google_bleu_metrics` methods, respectively. The resulting metrics are combined into a single DataFrame and returned.

__Viewing the Metrics__

We can display the resulting metrics DataFrame to see the evaluation scores for each model:
`metrics_df
`

This DataFrame shows the ROUGE and BLEU scores for each summarization model, allowing for an easy comparison of their performance.


__Note:__ It's crucial to understand that evaluating a summarization model based on just one text example from a testing dataset is not enough to determine its overall performance. The reason behind this is that the complexity of the text samples can vary significantly within a dataset. Some text samples might be easier for a model to summarize, while others could be more challenging, containing complex sentence structures, domain-specific jargon, or long and convoluted narratives.

To get a more comprehensive sense of a model's performance, it is essential to evaluate it on a larger number of samples from the testing dataset. By doing so, we can ensure that we're assessing the model's ability to handle a diverse set of inputs, including those that differ in complexity, topic, and structure. This approach provides a more reliable and generalizable evaluation of the model's performance, helping us to better understand its strengths and weaknesses.

In the next section of this notebook, we will expand our evaluation to include multiple text samples from the testing dataset. This will give us a better understanding of how well the model performs across a variety of inputs and allow us to make more informed decisions about its applicability to real-world use cases.

In [ ]:
evaluator = SummarizationMetrics()

metrics_df = evaluator.compute_sum_metric(summaries, reference)

metrics_df


## Compare all model summaries incl. baseline on CNN/Daily mail dataset

In [ ]:
print_summaries(summaries, reference)

# Use case arxiv: scientic paper summarization

In [ ]:
# Load the dataset
test_dataset = load_dataset("scientific_papers", "arxiv", split="test")
test_dataset.set_format(type='pandas', columns=['article', 'abstract'])


In [ ]:
# Convert to pandas DataFrame
df = test_dataset.to_pandas()

In [ ]:
sample = df.iloc[0]

In [ ]:
excerpt = 1500

print(f"\033[1mExcerpt of {excerpt} characters, total length of article: \
{len(sample['article'])}:\033[0m\n")

print(sample["article"][:excerpt])
print(f"\033[1m\n\nSummary (length: {len(sample['abstract'])}):\033[0m\n")
print(sample["abstract"])


In [ ]:
# Convert the pandas series to a numpy array
papers_np = test_dataset["article"].to_numpy()

papers_np.shape

In [ ]:
papers_np_1 = papers_np[:1]
papers_np_1.shape

In [ ]:
# Define a new function with the `words` argument fixed to 120
summarize_text_120 = partial(summarize_text, words=120)

# Parallelize the TextRank summarization
with Pool() as pool:
    summarized_articles = pool.map(summarize_text_120, papers_np_1)

In [ ]:
references_arxiv = test_dataset["abstract"][:1].tolist()

In [ ]:
references_arxiv = np.array(test_dataset["abstract"])

In [ ]:
references_arxiv_np = np.array(references_arxiv)
print(references_arxiv_np[:1])

In [ ]:
reference = references_arxiv[0]

In [ ]:
print(reference)

In [ ]:
print(summarized_articles[0])

In [ ]:
sample_text = sample[0]

In [ ]:
summaries_arxiv = {}
summaries_arxiv['TextRank (Baseline)'] = summarized_articles

## Generate summary from  <font color='red'>"BigBirdPegasus"</font> model

BigBird, is a sparse-attention based transformer which extends Transformer based models, such as BERT to much longer sequences. Moreover, BigBird comes along with a theoretical understanding of the capabilities of a complete transformer that the sparse model can handle. BigBird was introduced in this [paper](https://arxiv.org/abs/2007.14062).
<br><br>

__Model description__

BigBird relies on block sparse attention instead of normal attention (i.e. BERT's attention) and can handle sequences up to a length of 4096 at a much lower compute cost compared to BERT. It has achieved SOTA on various tasks involving very long sequences such as long documents summarization, question-answering with long contexts.

The models checkpoint is obtained after fine-tuning BigBirdPegasusForConditionalGeneration for summarization on arxiv dataset from scientific papers.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)

text = sample_text
inputs = tokenizer.encode_plus(text, return_tensors='pt',max_length=4096, truncation=True)
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction)

In [ ]:
summaries_arxiv['bigbird-pegasus'] = prediction

In [ ]:
summaries_arxiv['bigbird-pegasus'] = summaries_arxiv['bigbird-pegasus'][0]
summaries_arxiv['bigbird-pegasus']

## Generate summary from  <font color='red'>"Longformer"</font> model

First, it's important to note that the scores, such as ROUGE, for shorter text are generally higher than for longer text. This is because shorter text is easier to summarize accurately, and the metrics can capture the essence of the original text more effectively. In the case of the Longformer, its strong performance on longer text sequences is particularly noteworthy, as it demonstrates the model's ability to handle more complex summarization tasks effectively. This further emphasizes the strength of the Longformer and its potential for various text summarization applications.

Comparing the Longformer with other models on longer text, such as the arXiv dataset, can provide valuable insights into their performance. The arXiv dataset consists of scientific articles, which tend to be long and complex, making it an ideal benchmark for text summarization models.

For example, BART has achieved good results on the arXiv dataset. However, due to its limitations in handling longer text sequences, it may not be as effective as the Longformer. T5 has also demonstrated competitive performance on longer text summarization tasks, but it may still fall short when compared to the Longformer's capabilities.

Models like PEGASUS and BigBird, which have been designed with summarization-specific pre-training objectives, have shown promise on long text summarization tasks as well. However, as mentioned earlier, the Longformer outperforms these models despite not having a summarization-specific pre-training objective.

In summary, while other models have demonstrated varying levels of success on longer text summarization tasks such as the arXiv dataset, the Longformer stands out due to its ability to handle longer text sequences effectively and achieve high scores on various metrics, such as ROUGE.

The [official paper](https://arxiv.org/pdf/2004.05150.pdf) reports a new state-of-the-art __ROUGE-2 score of 19.62__ on the whole test dataset (WikiLingua dataset) which aligns very well with our observation here. LED significantly outperforms [PEGASUS](https://huggingface.co/transformers/model_doc/pegasus.html) and also slightly outperforms [BigBird](https://arxiv.org/abs/2007.14062) despite *PEGASUS* and
*BigBird* making use of a "summarization-specific" pre-training objective.

As a comparison, BART achieved a __ROUGE-2 score of 17.50__ on the arXiv dataset. It's important to note that these scores are for the standard BART model, which has limitations in handling longer text sequences.

In [ ]:
LONG_ARTICLE = sample_text
LONG_ARTICLE

In [ ]:
sample_text

In [ ]:

tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

input_ids = tokenizer(LONG_ARTICLE, padding="max_length", max_length=16384, return_tensors="pt", truncation=True).input_ids.to("cuda")
global_attention_mask = torch.zeros_like(input_ids)
# Put global attention on <s> token
global_attention_mask[:, 0] = 1

model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv", return_dict_in_generate=True).to("cuda")

sequences = model.generate(input_ids, global_attention_mask=global_attention_mask, max_length=512, num_beams=4).sequences

summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

In [ ]:
summaries_arxiv['Longformer'] = summary

In [ ]:
summaries_arxiv['Longformer'] = summaries_arxiv['Longformer'][0]
summaries_arxiv['Longformer']

In [ ]:
metrics_df_arxiv = evaluator.compute_sum_metric(summaries_arxiv, reference)

metrics_df_arxiv

## Summarizing a Small Dataset using Longformer and `generate_avg_summary`

In this section, we demonstrate how to use the `generate_avg_summary` function to generate summaries for a small dataset using the various models. The dataset in this example is assumed to be in the dataset_train variable, which is an instance of Hugging Face's Dataset class.

__Selecting a Small Range of Samples__

First, we select a small range of samples from the dataset. In this example, we select the first 10 samples:
`# Select a small range of samples
dataset_small = dataset_train.select(range(10))
`

__Converting the Dataset to a Pandas DataFrame__
Next, we convert the small dataset to a Pandas DataFrame, which provides an easier way to apply the generate_avg_summary function on each article:
`# Convert dataset to pandas dataframe
df = dataset_small.to_pandas()
`

__Generating Summaries using Longformer and our baseline, TextRank__

Now that we have a DataFrame containing the articles, we can apply the __generate_avg_summary__ function to each article to generate Longformer-based summaries.

You can adjust the `.select(range(10)` with different ranges to evaluate less or more articles.

In [ ]:
#tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")
# model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv", return_dict_in_generate=True).to("cuda")

def generate_avg_summary(article):
  input_ids = tokenizer(article, padding="max_length", max_length=16384, return_tensors="pt", truncation=True).input_ids.to("cuda")
  global_attention_mask = torch.zeros_like(input_ids)
  # Put global attention on <s> token
  global_attention_mask[:, 0] = 1

  sequences = model.generate(input_ids, global_attention_mask=global_attention_mask, max_length=512, num_beams=4).sequences
  summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

  return summary


In [ ]:
# Select a small range of samples
# See: https://huggingface.co/docs/datasets/v1.1.1/processing.html
dataset_small = test_dataset.select(range(10))

# Convert dataset to pandas dataframe
df = dataset_small.to_pandas()

# Generate summaries for each article
df["Longformer"] = df["article"].apply(generate_avg_summary)

# Print the summaries
print(df["Longformer"])

In [ ]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('google_bleu')

In [ ]:
# delete the list data format from each entry
df["Longformer"] = df['Longformer'].apply(lambda x: x[0])
df["Longformer"]

In [ ]:
# get 10 examples
papers_np_10 = papers_np[:10]
papers_np_10.shape

In [ ]:
# Define a new function with the `words` argument fixed to 120
summarize_text_120 = partial(summarize_text, words=120)

# Parallelize the TextRank summarization
with Pool() as pool:
    summarized_articles = pool.map(summarize_text_120, papers_np_10)

In [ ]:
df['TextRank (Baseline)'] = summarized_articles

In [ ]:
# convert to proper format to compute rouge metric
predictions = df["Longformer"].tolist()
references = df['abstract'].tolist()
results_longformer = rouge.compute(predictions=predictions,
                         references=references)
results_longformer_bleu = bleu.compute(predictions=predictions,
                         references=references)

In [ ]:
results_longformer_bleu


In [ ]:
# convert to proper format to compute rouge metric
predictions = df['TextRank (Baseline)'].tolist()
references = df['abstract'].tolist()
results_text_rank = rouge.compute(predictions=predictions,
                         references=references)
results_text_rank_bleu = bleu.compute(predictions=predictions,
                         references=references)

In [ ]:
# Get the value for TextRank
textRank_value = results_text_rank_bleu["google_bleu"]
longformer_value = results_longformer_bleu["google_bleu"]

In [ ]:
results_text_rank_bleu

In [ ]:
metrics = pd.DataFrame([results_text_rank, results_longformer], index=['TextRank (Baseline)', 'Longformer'])
metrics

In [ ]:
bleu_scores = [textRank_value, longformer_value]

# Using 'Address' as the column name
# and equating it to the list
metrics['google_bleu'] = bleu_scores
metrics

# Display Model Configuration

In [ ]:
# Load the configuration
config = BartConfig.from_pretrained("facebook/bart-large-cnn")

# Load the model with the specified configuration
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn", config=config)

# Print the model configuration
print(model.config)


In [ ]:
# Load the configuration
config = LEDConfig.from_pretrained("allenai/led-large-16384-arxiv")

# Load the model with the specified configuration
model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv", config=config)

# Print the model configuration
print(model.config)
